In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import os
from datetime import timedelta

In [26]:
# Fix path to correctly point to 'output/market_history' relative to the notebook
csv_dir = os.path.abspath('../output/market_history')

all_dfs = []

for i in range(1, 49):
    path = os.path.join(csv_dir, f'market_history_batch_{i}.csv')
    if os.path.exists(path) and os.path.getsize(path) > 0:
        df = pd.read_csv(path)
        all_dfs.append(df)
    else:
        print(f"Skipped missing or empty: {path}")

market_df = pd.concat(all_dfs, ignore_index=True)
market_df['date'] = pd.to_datetime(market_df['date'])  # Convert to datetime

item_map = pd.read_csv('../output/all_item_type_ids.csv')



In [27]:
# Merge item names into market history
full_df = market_df.merge(item_map, on='type_id')

# Unique sorted item names for dropdown
item_names = sorted(full_df['item_name'].unique())

# Dropdown widget with search
import ipywidgets as widgets
from IPython.display import display
from datetime import timedelta
import matplotlib.pyplot as plt

dropdown = widgets.Combobox(
    placeholder='Search item...',
    options=item_names,
    description='Item:',
    ensure_option=True,
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

def plot_item(change):
    output.clear_output()
    selected_item = change['new']
    
    if selected_item not in item_names:
        return

    with output:
        today = pd.Timestamp.today()
        one_year_ago = today - timedelta(days=365)

        filtered = full_df[
            (full_df['item_name'] == selected_item) &
            (full_df['date'] >= one_year_ago)
        ].sort_values(by='date')

        if filtered.empty:
            print("No data available for this item in the last year.")
            return

        plt.figure(figsize=(12, 6))
        plt.plot(filtered['date'], filtered['average'], label='Average Price', linewidth=2)
        plt.fill_between(filtered['date'], filtered['low'], filtered['high'], alpha=0.3, label='Price Range')
        plt.title(f"{selected_item} - Market History (Last 365 Days)")
        plt.xlabel('Date')
        plt.ylabel('ISK')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

dropdown.observe(plot_item, names='value')
display(dropdown, output)


KeyError: 'item_name'